In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import os

load_dotenv()
llm = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen-plus",  # 此处以qwen-plus为例，您可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
)
# 提示词模板
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant, willing to answer as many questions as possible. The chat history provided contains relevant information about the user you are conversing with."),
    MessagesPlaceholder(variable_name="history", optional=True),
    ("human", "{input}")
])
chain = prompt_template | llm
resp = chain.invoke({"input": "what is ATP?"})
print(resp)

content='ATP stands for **adenosine triphosphate**. It is a molecule that serves as the primary **energy currency** of cells in all living organisms.\n\n### Structure:\nATP is composed of:\n- **Adenine** (a nitrogenous base),\n- **Ribose** (a five-carbon sugar), and\n- **Three phosphate groups** linked in a chain.\n\n### Function:\n- ATP stores and transfers energy within cells.\n- When a cell needs energy, it breaks the bond between the second and third phosphate groups through a process called **hydrolysis**, converting ATP into **ADP (adenosine diphosphate)** and an inorganic phosphate (Pi), releasing energy in the process:\n  \n  **ATP → ADP + Pi + energy**\n\n- This energy powers essential cellular processes such as:\n  - Muscle contraction\n  - Nerve impulse propagation\n  - Chemical synthesis (e.g., building proteins or DNA)\n  - Active transport across cell membranes\n\n### Production:\nATP is produced primarily through:\n- **Cellular respiration** in mitochondria (in eukaryote

In [ ]:
# 存储联调记录（可选存储方式: 内存，DB，redis）,下面先使用内存方式
# key为session_id
from langchain_core.chat_history import InMemoryChatMessageHistory, SQLChatMessageHistory
store = {} 

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 数据库的版本
# def get_session_history(session_id: str) -> list:
#     return SQLChatMessageHistory(
#         session_id=session_id,
#         connection_string="sqlite:///chat_history.db",
#     )

In [13]:
# 创建带历史记录功能的处理链
from langchain_core.runnables.history import RunnableWithMessageHistory

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

res = chain_with_history.invoke(  # noqa: T201
                {"input": "What does cosine mean?"},
                config={"configurable": {"session_id": "user123"}}
            )
print(res.content)

**Cosine** is a fundamental trigonometric function in mathematics, often abbreviated as **cos**. It relates the angles of a right triangle to the ratios of its sides.

### In a Right Triangle:
For a given **acute angle** (let’s call it θ) in a right triangle:

> **cos(θ) = adjacent side / hypotenuse**

- The **adjacent side** is the side next to the angle θ (but not the hypotenuse).
- The **hypotenuse** is the longest side, opposite the right angle.

### On the Unit Circle:
Cosine can also be defined using the **unit circle** (a circle with radius 1 centered at the origin):

> For an angle θ measured from the positive x-axis, **cos(θ)** is the **x-coordinate** of the point where the terminal side of the angle intersects the unit circle.

This definition extends cosine to **any angle**, not just those between 0° and 90°.

### Key Properties:
- **Range**: Cosine values always lie between **–1 and 1**.
- **Periodicity**: Cosine is periodic with a period of **360°** (or **2π radians**), me

In [14]:
res = chain_with_history.invoke(  # noqa: T201
                {"input": "What's its inverse"},
                config={"configurable": {"session_id": "user123"}}
            )
print(res.content)

The **inverse of the cosine function** is called the **arccosine** (or **inverse cosine**) function, and it's commonly written as:

- **arccos(x)**
- or **cos⁻¹(x)**  (note: this does *not* mean 1/cos(x)—that’s a common confusion!)

---

### What does it do?

While the **cosine** function takes an **angle** and gives you a **ratio** (adjacent/hypotenuse), the **arccosine** does the reverse:

> **arccos(ratio) = angle**

So if you know the cosine of an angle, arccos tells you **what that angle is**.

---

### Example:

You know that:  
**cos(60°) = 0.5**

Then:  
**arccos(0.5) = 60°**  
(or **π/3 radians**, if you're using radians)

---

### Domain and Range:

Because cosine isn't one-to-one over all real numbers (it repeats), we **restrict its domain** to make the inverse well-defined.

- **Domain of arccos(x):**  
  \( -1 \leq x \leq 1 \)  
  (because cosine only outputs values between –1 and 1)

- **Range of arccos(x):**  
  \( 0 \leq \text{arccos}(x) \leq \pi \) radians  
  (or **0°

In [ ]:
# 修剪聊天上下文, 保留最近的n条消息，将之前的所有消息形成摘要
def summarize_messages(cur_input):
    session_id = cur_input['config']['configurable']['session_id']
    if not session_id:
        raise ValueError("session_id is required")
    chat_history = get_session_history(session_id)
    stored_messages = chat_history.messages

def trim_chat_history(chat_history: list, n: int = 5) -> list:
    if len(chat_history) <= n:
        return chat_history
    return chat_history[-n:]


4
